# A simple MLP example for MNIST dataset

In [1]:
import torch
import torchvision 
import torch.nn.functional as F  
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms
from torch import optim  # For optimizers SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm # For nice progress bar!

build a network

In [2]:
class NN(nn.Module):
    def __init__(self , input_size , num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size , 50)
        self.fc2 = nn.Linear(50 , num_classes)
    ##########################
    def forward(self , x ):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

set device

In [3]:
target_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameter

In [4]:
input_size = 784
num_classes = 10
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

Load Training and Test data. Usually need to write dataloaber ourselves.

In [5]:

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=False)
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Initialize network

In [7]:
model = NN(input_size=input_size, num_classes=num_classes).to(target_device)

In [8]:
# Loss and optimizer

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the Network

In [11]:
for epoch in range(num_epochs):
    for batch_idx , ( data, targets ) in enumerate( tqdm( train_loader ) ):
        # Get data to cuda if possible
        data = data.to( device = target_device )
        targets = targets.to( device = target_device )

        # Get to correct shape
        # data.shape[0] usually is the number of examples in a batch
        # in this case, just set each sample to a long bar
        data = data.reshape(data.shape[0], -1)

        # forward
        predictions = model(data)
        #targets are true labels
        loss = loss_func(predictions , targets)

        # backward
        optimizer.zero_grad() # set the gradients to zero, so that we are ready for the next loop
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:14<00:00, 64.61it/s]


In [12]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=target_device)
            y = y.to(device=target_device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.70
Accuracy on test set: 97.31
